In [1]:
import import_ipynb #allows access to import other notebooks
from PreprocessLib import cleanText, removeSingleLists

importing Jupyter notebook from PreprocessLib.ipynb


In [2]:
#processes files for Doc2Vec
def process_file_doc(a_file):
    string_without_line_breaks = ""
    for line in a_file:
        stripped_line = line.rstrip() + " "
        string_without_line_breaks += stripped_line
    a_file.close()
    sentences = string_without_line_breaks

    cleanedFile = cleanText(sentences, False) #cleanText is found in PreprocessLib.ipynb
    return cleanedFile

In [3]:
longFile = open("./other-sources/Longus.txt", "r", encoding="utf8")
markFile = open("./other-sources/Mark.txt", "r", encoding="utf8")
mathFile = open("./other-sources/Mathew.txt", "r", encoding="utf8")
ignatFile = open("./other-sources/Ignatius-4.txt", "r", encoding="utf8")
testFile = open("./other-sources/Ignatius-3books.txt", "r", encoding="utf8")
#processes files for D2V
long = process_file_doc(longFile)
mark = process_file_doc(markFile)
math = process_file_doc(mathFile)
ignat = process_file_doc(ignatFile)
test = process_file_doc(testFile)

In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#places the texts in a list with tags for each author
docs = []
docs.append(TaggedDocument(words=long, tags=["Longus"]))
docs.append(TaggedDocument(words=mark, tags=["Mark"]))
docs.append(TaggedDocument(words=math, tags=["Mathew"]))
docs.append(TaggedDocument(words=ignat, tags=["Ignatius"]))

In [7]:
from scipy import spatial
docVecs = {}
#computes cosine similarity of documents with test document 20 times (to account for variation in infer_vector)
for i in range(20):
    docVecs['Test'] = model.infer_vector(test.split())
    print(model.docvecs.most_similar(positive=[docVecs['Test']]))

[('Ignatius', 0.4790303707122803), ('Mathew', 0.06569336354732513), ('Mark', -0.010337963700294495), ('Longus', -0.13260343670845032)]
[('Ignatius', 0.4639646112918854), ('Mathew', 0.09112807363271713), ('Mark', 0.008441105484962463), ('Longus', -0.12055673450231552)]
[('Ignatius', 0.4580806493759155), ('Mathew', 0.07400600612163544), ('Mark', -0.005612591281533241), ('Longus', -0.13014990091323853)]
[('Ignatius', 0.4768056869506836), ('Mathew', 0.08016101270914078), ('Mark', 0.004031240940093994), ('Longus', -0.1472066044807434)]
[('Ignatius', 0.47071006894111633), ('Mathew', 0.08696802705526352), ('Mark', 0.011553622782230377), ('Longus', -0.1378754824399948)]
[('Ignatius', 0.4718053936958313), ('Mathew', 0.06752533465623856), ('Mark', -0.01105906069278717), ('Longus', -0.12178531289100647)]
[('Ignatius', 0.46674612164497375), ('Mathew', 0.08256402611732483), ('Mark', 0.004893852397799492), ('Longus', -0.13722050189971924)]
[('Ignatius', 0.4710613787174225), ('Mathew', 0.096516810357

In [178]:
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import ParameterGrid
from scipy import spatial

parm_dict = {'alpha':(0.035,0.025,0.015), 'min_count':(1, 2, 4), 'vector_size':(10, 100, 200), 
            'epochs':(10, 100, 2000), 'dm':(0,1), 'steps':(10, 20)}
#runs grid search of the associated hyperparameters, computes cosine similarity, and prints the results
def runGrid():
    largestSim = -1
    bestIteration = []
    param_grid = ParameterGrid(parm_dict)
    for dict_ in param_grid:
        print("---------------------------------------------------")
        print(dict_)
        model = gensim.models.doc2vec.Doc2Vec(min_count=dict_["min_count"], alpha=dict_["alpha"],
                            epochs=dict_["epochs"], dm=dict_["dm"], vector_size=dict_["vector_size"])
        model.build_vocab(docs)
        model.train(docs, total_examples=model.corpus_count, epochs=1000)
        
        docVecs = {}
        docVecs['Test'] = model.infer_vector(test.split(), alpha=dict_["alpha"], epochs=dict_["epochs"], steps=dict_["steps"])
        mostSim = model.docvecs.most_similar(positive=[docVecs['Test']])
        print(model.docvecs.most_similar(positive=[docVecs['Test']]))
        tempVal = 0
     
        for val in mostSim:
            if (val[0] == "Ignatius"):
                tempVal = val[1]
        if (tempVal > largestSim):
            bestIteration = mostSim
            largestSim = tempVal
    print(bestIteration)
    print(largestSim)

In [179]:
runGrid()
#best iteration:
#{'alpha': 0.035, 'dm': 0, 'epochs': 10, 'min_count': 1, 'steps': 10, 'vector_size': 200}
#[('Ignatius', 0.8577240705490112), ('Mark', 0.4368272125720978), ('Mathew', 0.2611377239227295), ('Longus', 0.025741420686244965)]

---------------------------------------------------
{'alpha': 0.035, 'dm': 0, 'epochs': 10, 'min_count': 1, 'steps': 10, 'vector_size': 10}
[('Ignatius', 0.8329225778579712), ('Mark', 0.4069109857082367), ('Mathew', 0.22481873631477356), ('Longus', -0.04066843539476395)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 0, 'epochs': 10, 'min_count': 1, 'steps': 10, 'vector_size': 100}
[('Ignatius', 0.8459982872009277), ('Mark', 0.39376747608184814), ('Mathew', 0.22206008434295654), ('Longus', -0.045539140701293945)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 0, 'epochs': 10, 'min_count': 1, 'steps': 10, 'vector_size': 200}
[('Ignatius', 0.8577240705490112), ('Mark', 0.4368272125720978), ('Mathew', 0.2611377239227295), ('Longus', 0.025741420686244965)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 0, 'epochs': 10, 'min_count': 1, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.8332536816596985), ('Mark

[('Ignatius', 0.7720457911491394), ('Mark', 0.34943071007728577), ('Mathew', 0.10000243782997131), ('Longus', -0.21073859930038452)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 0, 'epochs': 100, 'min_count': 4, 'steps': 10, 'vector_size': 100}
[('Ignatius', 0.6878513097763062), ('Mark', 0.34307003021240234), ('Mathew', 0.10234864056110382), ('Longus', -0.15985515713691711)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 0, 'epochs': 100, 'min_count': 4, 'steps': 10, 'vector_size': 200}
[('Ignatius', 0.7583485841751099), ('Mark', 0.3180438280105591), ('Mathew', 0.14996013045310974), ('Longus', -0.13427075743675232)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 0, 'epochs': 100, 'min_count': 4, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.7696009874343872), ('Mark', 0.3542744219303131), ('Mathew', 0.10911184549331665), ('Longus', -0.18951961398124695)]
---------------------------------------------

[('Mathew', 0.22689101099967957), ('Ignatius', 0.22524411976337433), ('Mark', 0.18092483282089233), ('Longus', -0.06289719045162201)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 1, 'epochs': 10, 'min_count': 2, 'steps': 10, 'vector_size': 100}
[('Ignatius', 0.09214109927415848), ('Mark', -0.022634092718362808), ('Mathew', -0.06539560854434967), ('Longus', -0.08062754571437836)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 1, 'epochs': 10, 'min_count': 2, 'steps': 10, 'vector_size': 200}
[('Ignatius', 0.10299909114837646), ('Mark', -0.012011498212814331), ('Mathew', -0.03356432169675827), ('Longus', -0.038865577429533005)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 1, 'epochs': 10, 'min_count': 2, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.26389598846435547), ('Mathew', 0.16470077633857727), ('Mark', 0.13712835311889648), ('Longus', -0.023566782474517822)]
----------------------------------

[('Ignatius', 0.5572818517684937), ('Mark', 0.053855761885643005), ('Mathew', -0.00013327598571777344), ('Longus', -0.33656445145606995)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 1, 'epochs': 2000, 'min_count': 1, 'steps': 10, 'vector_size': 100}
[('Ignatius', 0.07990910112857819), ('Mark', 0.07531528919935226), ('Mathew', 0.017050527036190033), ('Longus', -0.07597219944000244)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 1, 'epochs': 2000, 'min_count': 1, 'steps': 10, 'vector_size': 200}
[('Mark', 0.08381365984678268), ('Ignatius', 0.072968028485775), ('Mathew', 0.020016584545373917), ('Longus', -0.06440854072570801)]
---------------------------------------------------
{'alpha': 0.035, 'dm': 1, 'epochs': 2000, 'min_count': 1, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.6680582761764526), ('Mathew', 0.05454196035861969), ('Mark', 0.004401318728923798), ('Longus', -0.16051623225212097)]
--------------------------------

[('Ignatius', 0.8653614521026611), ('Mark', 0.40996238589286804), ('Mathew', 0.22612762451171875), ('Longus', -0.00756286084651947)]
---------------------------------------------------
{'alpha': 0.025, 'dm': 0, 'epochs': 10, 'min_count': 4, 'steps': 10, 'vector_size': 100}
[('Ignatius', 0.8352875709533691), ('Mark', 0.4084480106830597), ('Mathew', 0.21738582849502563), ('Longus', 0.02140391431748867)]
---------------------------------------------------
{'alpha': 0.025, 'dm': 0, 'epochs': 10, 'min_count': 4, 'steps': 10, 'vector_size': 200}
[('Ignatius', 0.8677910566329956), ('Mark', 0.46536827087402344), ('Mathew', 0.2551683187484741), ('Longus', 0.0990864634513855)]
---------------------------------------------------
{'alpha': 0.025, 'dm': 0, 'epochs': 10, 'min_count': 4, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.8727171421051025), ('Mark', 0.4299599826335907), ('Mathew', 0.23182865977287292), ('Longus', 0.025489293038845062)]
---------------------------------------------------


...

[('Ignatius', 0.3119315505027771), ('Mathew', 0.06553179770708084), ('Mark', 0.020190946757793427), ('Longus', -0.1111779659986496)]
---------------------------------------------------
{'alpha': 0.025, 'dm': 1, 'epochs': 100, 'min_count': 1, 'steps': 10, 'vector_size': 100}
[('Ignatius', 0.06980106979608536), ('Mark', 0.021955788135528564), ('Mathew', -0.025102511048316956), ('Longus', -0.10196800529956818)]
---------------------------------------------------
{'alpha': 0.025, 'dm': 1, 'epochs': 100, 'min_count': 1, 'steps': 10, 'vector_size': 200}
[('Ignatius', 0.0685766339302063), ('Mark', 0.038076817989349365), ('Mathew', -0.015115097165107727), ('Longus', -0.10383807867765427)]
---------------------------------------------------
{'alpha': 0.025, 'dm': 1, 'epochs': 100, 'min_count': 1, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.30850446224212646), ('Mathew', -0.023945778608322144), ('Mark', -0.026492688804864883), ('Longus', -0.34619447588920593)]
-------------------------------

[('Ignatius', 0.479826956987381), ('Mark', -0.03686462342739105), ('Mathew', -0.04007422551512718), ('Longus', -0.216334730386734)]
---------------------------------------------------
{'alpha': 0.025, 'dm': 1, 'epochs': 2000, 'min_count': 4, 'steps': 10, 'vector_size': 100}
[('Mark', 0.07374647259712219), ('Ignatius', 0.0672600045800209), ('Mathew', 0.03777360916137695), ('Longus', -0.09025858342647552)]
---------------------------------------------------
{'alpha': 0.025, 'dm': 1, 'epochs': 2000, 'min_count': 4, 'steps': 10, 'vector_size': 200}
[('Mark', 0.0743546187877655), ('Ignatius', 0.06786669045686722), ('Mathew', 0.03569100797176361), ('Longus', -0.07364651560783386)]
---------------------------------------------------
{'alpha': 0.025, 'dm': 1, 'epochs': 2000, 'min_count': 4, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.5939212441444397), ('Mathew', -0.0014713630080223083), ('Mark', -0.05372142791748047), ('Longus', -0.15280629694461823)]
-------------------------------------

[('Ignatius', 0.7815656065940857), ('Mark', 0.3744507431983948), ('Mathew', 0.08300039172172546), ('Longus', -0.16517357528209686)]
---------------------------------------------------
{'alpha': 0.015, 'dm': 0, 'epochs': 100, 'min_count': 2, 'steps': 10, 'vector_size': 200}
[('Ignatius', 0.8327819108963013), ('Mark', 0.3796876072883606), ('Mathew', 0.1203284040093422), ('Longus', -0.05575236678123474)]
---------------------------------------------------
{'alpha': 0.015, 'dm': 0, 'epochs': 100, 'min_count': 2, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.8296862840652466), ('Mark', 0.3741096258163452), ('Mathew', 0.10000547021627426), ('Longus', -0.1804618239402771)]
---------------------------------------------------
{'alpha': 0.015, 'dm': 0, 'epochs': 100, 'min_count': 2, 'steps': 20, 'vector_size': 100}
[('Ignatius', 0.8213852047920227), ('Mark', 0.3859056830406189), ('Mathew', 0.10376875847578049), ('Longus', -0.08584759384393692)]
-------------------------------------------------

[('Ignatius', 0.10135644674301147), ('Longus', -0.007938399910926819), ('Mark', -0.01275646686553955), ('Mathew', -0.02514253556728363)]
---------------------------------------------------
{'alpha': 0.015, 'dm': 1, 'epochs': 10, 'min_count': 1, 'steps': 10, 'vector_size': 200}
[('Ignatius', 0.08960022032260895), ('Mark', -0.0004578661173582077), ('Mathew', -0.014708754606544971), ('Longus', -0.05073738098144531)]
---------------------------------------------------
{'alpha': 0.015, 'dm': 1, 'epochs': 10, 'min_count': 1, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.2093079686164856), ('Mathew', 0.11922553181648254), ('Mark', 0.08757050335407257), ('Longus', 0.021678462624549866)]
---------------------------------------------------
{'alpha': 0.015, 'dm': 1, 'epochs': 10, 'min_count': 1, 'steps': 20, 'vector_size': 100}
[('Ignatius', 0.07432471215724945), ('Mark', -0.005529329180717468), ('Mathew', -0.011871572583913803), ('Longus', -0.03750942274928093)]
-------------------------------

[('Ignatius', 0.05905294418334961), ('Mark', 0.01440802775323391), ('Mathew', -0.015597730875015259), ('Longus', -0.10289560258388519)]
---------------------------------------------------
{'alpha': 0.015, 'dm': 1, 'epochs': 100, 'min_count': 4, 'steps': 10, 'vector_size': 200}
[('Ignatius', 0.06534141302108765), ('Mark', 0.02984705939888954), ('Mathew', -0.018582304939627647), ('Longus', -0.1089794933795929)]
---------------------------------------------------
{'alpha': 0.015, 'dm': 1, 'epochs': 100, 'min_count': 4, 'steps': 20, 'vector_size': 10}
[('Ignatius', 0.2755531668663025), ('Mathew', 0.04654768854379654), ('Mark', 0.035409148782491684), ('Longus', -0.0669422522187233)]
---------------------------------------------------
{'alpha': 0.015, 'dm': 1, 'epochs': 100, 'min_count': 4, 'steps': 20, 'vector_size': 100}
[('Ignatius', 0.061248552054166794), ('Mark', 0.007588006556034088), ('Mathew', -0.03596890717744827), ('Longus', -0.12035167962312698)]
----------------------------------